<a href='https://github.com/SeWonKwon' ><div> <img src ='https://slid-capture.s3.ap-northeast-2.amazonaws.com/public/image_upload/6556674324ed41a289a354258718280d/964e5a8b-75ad-41fc-ae75-0ca66d06fbc7.png' align='left' /> </div></a>

# 지도학습 - IMDB 영화평

# data_load & get info

data source : https://www.kaggle.com/c/word2vec-nlp-tutorial/data

In [1]:
import gdown
import os
import pandas as pd
def get_dataset():

    google_path = 'https://drive.google.com/uc?id='
    file_id_train = '1fOpIndzsAwzrBbuJMjov_alDmgGUsP0d'
    file_id_test = '1OQuaDbTWoILlnYMfYjfkMnCzeR1xU5gx'

    gdown.download(google_path+file_id_train,'TrainData.tsv',quiet=False)
#     gdown.download(google_path+file_id_test,'TestData.tsv',quiet=False)
    
    X = pd.read_csv('TrainData.tsv', header=0, sep='\t', quoting=3)
#     X_test = pd.read_csv('TestData.tsv', header=0, sep='\t', quoting=3)

    os.remove('TrainData.tsv')
#     os.remove('TestData.tsv')
    
    return X

df = get_dataset()

Downloading...
From: https://drive.google.com/uc?id=1fOpIndzsAwzrBbuJMjov_alDmgGUsP0d
To: C:\Users\bigne\OneDrive\WorkJ\Machine Learning\NLP\TrainData.tsv
33.6MB [00:00, 36.1MB/s]


In [2]:
def get_df_info(df, short=False):
    print('데이터 세트의 Shape:', df.shape)
    print('\n전체 피처의 dtype\n', df.dtypes.value_counts())
    isnull_series = df.isnull().sum()
    null_column_count = df.isnull().sum()[df.isnull().sum()>0]
    df_null = pd.DataFrame({'dtype':df.dtypes[null_column_count.index], 'count':df.isnull().sum()[df.isnull().sum()>0]})
    print('\n########################\nNull count df\n')
    if short:
        display(df_null.sort_values('count',ascending=False).head())
        display(df.describe().T.head())
    else:
        display(df_null.sort_values('count',ascending=False))
        display(df.describe().T)

In [3]:
get_df_info(df)

데이터 세트의 Shape: (25000, 3)

전체 피처의 dtype
 object    2
int64     1
dtype: int64

########################
Null count df



,dtype,count


,count,mean,std,min,25%,50%,75%,max
sentiment,25000.0,0.5,0.50001,0.0,0.0,0.5,1.0,1.0


In [4]:
df.columns

Index(['id', 'sentiment', 'review'], dtype='object')

In [5]:
df['sentiment'].value_counts()

1    12500
0    12500
Name: sentiment, dtype: int64

# cleaning

In [6]:
import re 

df['review'] = df['review'].str.replace('<br />', ' ')

df['review'] = df['review'].apply( lambda x : re.sub('[^a-zA-Z]', ' ', x))


In [7]:
from sklearn.model_selection import train_test_split

class_df = df['sentiment']
feature_df = df.drop(['id', 'sentiment'], axis=1, inplace= False)

X_train, X_test, y_train, y_test = train_test_split(feature_df, class_df, 
                                                    test_size=0.3, random_state=156)

print(X_train.shape, X_test.shape)
print(y_train.value_counts())

(17500, 1) (7500, 1)
0    8820
1    8680
Name: sentiment, dtype: int64


In [8]:
y_train.shape

(17500,)

In [9]:
X_train.shape

(17500, 1)

In [10]:
X_train['review'].shape

(17500,)

In [11]:
X_test.columns

Index(['review'], dtype='object')

In [12]:
X_test.shape

(7500, 1)

In [13]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

pipeline = Pipeline([
    ('cnt_vect', CountVectorizer(stop_words='english', ngram_range=(1, 2))),
    ('lr_clf', LogisticRegression(C=10))
                    ])

pipeline.fit(X_train['review'], y_train)
pred = pipeline.predict(X_test['review'])
pred_probs = pipeline.predict_proba(X_test['review'])[:, 1]

print('예측 정확도 {0:.4f}, ROC-AUC는 {1:.4f}'.format(accuracy_score(y_test, pred),
                                                roc_auc_score(y_test, pred_probs)))

C:\Users\bigne\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


예측 정확도 0.8860, ROC-AUC는 0.9503


위의 코드에서 X_train대신에 X_train['review']를 하는 이유는 아무래도 vectorize 특성 때문인거 같다. shape이 (17500,1) vs (17500,) 인데 2차원 vs 1차원 인듯 합니다. 

y_train이 1차원이기 때문에 같은 차원수를 가져야 vectorize가 되는거 같다. 

## unsupervised Sentiment Analysis

* SentiWordNet : 긍정 감성 지수, 부정 감성 지수, 객관성 지수
* VADER : 소셜 미디어의 텍스트에 대한 감성 분석을 제공
* Pattern : 예측 성능 측면에서 가장 주목받는 패키지 이지만, 파이썬 3.X 버전에서는 호환이 안됨

In [14]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\bigne\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\bigne\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     C:\Users\bigne\AppData\Roaming\nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to
[nltk_data]    |     C:\Users\bigne\AppData\Roaming\nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     C:\Users\bigne\AppData\Roaming\nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to
[nltk_data]    |     C:\Users\b

[nltk_data]    |   Package ptb is already up-to-date!
[nltk_data]    | Downloading package product_reviews_1 to
[nltk_data]    |     C:\Users\bigne\AppData\Roaming\nltk_data...
[nltk_data]    |   Package product_reviews_1 is already up-to-date!
[nltk_data]    | Downloading package product_reviews_2 to
[nltk_data]    |     C:\Users\bigne\AppData\Roaming\nltk_data...
[nltk_data]    |   Package product_reviews_2 is already up-to-date!
[nltk_data]    | Downloading package pros_cons to
[nltk_data]    |     C:\Users\bigne\AppData\Roaming\nltk_data...
[nltk_data]    |   Package pros_cons is already up-to-date!
[nltk_data]    | Downloading package qc to
[nltk_data]    |     C:\Users\bigne\AppData\Roaming\nltk_data...
[nltk_data]    |   Package qc is already up-to-date!
[nltk_data]    | Downloading package reuters to
[nltk_data]    |     C:\Users\bigne\AppData\Roaming\nltk_data...
[nltk_data]    |   Package reuters is already up-to-date!
[nltk_data]    | Downloading package rte to
[nltk_data]  

[nltk_data]    |   Package word2vec_sample is already up-to-date!
[nltk_data]    | Downloading package panlex_swadesh to
[nltk_data]    |     C:\Users\bigne\AppData\Roaming\nltk_data...
[nltk_data]    |   Package panlex_swadesh is already up-to-date!
[nltk_data]    | Downloading package mte_teip5 to
[nltk_data]    |     C:\Users\bigne\AppData\Roaming\nltk_data...
[nltk_data]    |   Package mte_teip5 is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\bigne\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\bigne\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package perluniprops to
[nltk_data]    |     C:\Users\bigne\AppData\Roamin

True

In [15]:
from nltk.corpus import wordnet as wn

term = 'present'

synsets = wn.synsets(term)
print('synsets() 반환 type:', type(synsets))
print('synsets() 반환 값 갯수:', len(synsets))
print('synsets() 반환 값:', synsets)

synsets() 반환 type: <class 'list'>
synsets() 반환 값 갯수: 18
synsets() 반환 값: [Synset('present.n.01'), Synset('present.n.02'), Synset('present.n.03'), Synset('show.v.01'), Synset('present.v.02'), Synset('stage.v.01'), Synset('present.v.04'), Synset('present.v.05'), Synset('award.v.01'), Synset('give.v.08'), Synset('deliver.v.01'), Synset('introduce.v.01'), Synset('portray.v.04'), Synset('confront.v.03'), Synset('present.v.12'), Synset('salute.v.06'), Synset('present.a.01'), Synset('present.a.02')]


In [16]:
type(synsets)

list

In [17]:
i=0
for synset in synsets[:5]:
    i+=1
    print('##### Synset name : ', synset.name(),'#####', i)
    print('POS :',synset.lexname())
    print('Definition:',synset.definition())
    print('Lemmas:',synset.lemma_names())

##### Synset name :  present.n.01 ##### 1
POS : noun.time
Definition: the period of time that is happening now; any continuous stretch of time including the moment of speech
Lemmas: ['present', 'nowadays']
##### Synset name :  present.n.02 ##### 2
POS : noun.possession
Definition: something presented as a gift
Lemmas: ['present']
##### Synset name :  present.n.03 ##### 3
POS : noun.communication
Definition: a verb tense that expresses actions or states at the time of speaking
Lemmas: ['present', 'present_tense']
##### Synset name :  show.v.01 ##### 4
POS : verb.perception
Definition: give an exhibition of to an interested audience
Lemmas: ['show', 'demo', 'exhibit', 'present', 'demonstrate']
##### Synset name :  present.v.02 ##### 5
POS : verb.communication
Definition: bring forward and present to the mind
Lemmas: ['present', 'represent', 'lay_out']


In [18]:
# synset 객체를 단어별로 생성합니다. 

tree = wn.synset('tree.n.01')
lion = wn.synset('lion.n.01')
tiger = wn.synset('tiger.n.02')
cat = wn.synset('cat.n.01')
dog = wn.synset('dog.n.01')

entities = [tree, lion, tiger, cat, dog]

entity_names = [entity.name().split('.')[0] for entity in entities]

In [19]:
import pandas as pd

similarities = []

for entity in entities:
    similarity = [ round(entity.path_similarity(compared_entity),3) for compared_entity in entities]
    similarities.append(similarity)

similarity_df = pd.DataFrame(data=similarities, columns=entity_names, index=entity_names)
similarity_df

,tree,lion,tiger,cat,dog
tree,1.000,0.071,0.071,0.077,0.125
lion,0.071,1.000,0.333,0.250,0.167
tiger,0.071,0.333,1.000,0.250,0.167
cat,0.077,0.250,0.250,1.000,0.200
dog,0.125,0.167,0.167,0.200,1.000


In [20]:
import nltk
from nltk.corpus import sentiwordnet as swn

senti_synsets = list(swn.senti_synsets('slow'))
print('senti_synsets() 반환 type :', type(senti_synsets))
print('senti_synsets() 반환 값 갯수:', len(senti_synsets))
print('senti_synsets() 반환 값 :', senti_synsets)


senti_synsets() 반환 type : <class 'list'>
senti_synsets() 반환 값 갯수: 11
senti_synsets() 반환 값 : [SentiSynset('decelerate.v.01'), SentiSynset('slow.v.02'), SentiSynset('slow.v.03'), SentiSynset('slow.a.01'), SentiSynset('slow.a.02'), SentiSynset('dense.s.04'), SentiSynset('slow.a.04'), SentiSynset('boring.s.01'), SentiSynset('dull.s.08'), SentiSynset('slowly.r.01'), SentiSynset('behind.r.03')]


In [21]:
import nltk
from nltk.corpus import sentiwordnet as swn

father = swn.senti_synset('father.n.01')
print('father 긍정감성 지수: ', father.pos_score())
print('father 부정감성 지수: ', father.neg_score())
print('father 객관성 지수: ', father.obj_score())
print('\n')
fabulous = swn.senti_synset('fabulous.a.01')
print('fabulous 긍정감성 지수: ',fabulous .pos_score())
print('fabulous 부정감성 지수: ',fabulous .neg_score())
print('father 객관성 지수: ', fabulous.obj_score())

father 긍정감성 지수:  0.0
father 부정감성 지수:  0.0
father 객관성 지수:  1.0


fabulous 긍정감성 지수:  0.875
fabulous 부정감성 지수:  0.125
father 객관성 지수:  0.0


SentiWordNet Lexicon 기반 - IMDB 영화 감상평 잠성 분석

1. 문서(Document)를 문장(Sentence) 단위로 분해

2. 다시 문장을 단어(Word) 단위로 토큰화하고 품사 태깅

3. 품사 태깅된 단어 기반으로 synet 객체와 senti_synset 객체를 생성

4. Senti_synset에서 긍정 감성/부정 감성 지수를 구하고 이를 모두 합산해 특정 임계치 값 이상일 때 긍정 감성으로, 그렇지 않을 때는 부정 감성으로 결정

In [22]:
from nltk.corpus import wordnet as wn

# 간단한 NTLK PennTreebank Tag를 기반으로 WordNet 기반의 품사 Tag로 변환

def penn_to_wn(tag): # NTLK PennTreebank Tag to WordNet 
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    

In [23]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag

def swn_polarity(text):
    # 감성 지수 초기화
    sentiment = 0.0
    tokens_count = 0 
    
    lemmatizer = WordNetLemmatizer()
    raw_sentences = sent_tokenize(text)
    
    for raw_sentence in raw_sentences:
        # NTLK 기반의 품사 태깅 문장 추출
        tagged_sentence = pos_tag(word_tokenize(raw_sentence))
        for word, tag in tagged_sentence:
            
            # WordNet 기반 품사 태깅과 어근 추출
            wn_tag = penn_to_wn(tag)
            if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):
                continue
            lemma = lemmatizer.lemmatize(word, pos=wn_tag)
            if not lemma:
                continue
            # 어근을 추출한 단어와 WordNet 기반 품사 태깅을 입력해 Synset 객체를 생성.
            if not synsets:
                continue
            # sentiwordnet의 감성 단어 분석으로 감성 synset 추출
            # 모든 단어에 대해 긍정 감성 지수는 +로 부정 감성지수는 -로 합산해서 감성 분석
            
            synset = synsets[0]
            swn_synset = swn.senti_synset(synset.name())
            sentiment += (swn_synset.pos_score() - swn_synset.neg_score())
            tokens_count += 1
    if not tokens_count:
        return 0
    
    # 총 score가 0 이상일 경우 긍정(Positive) 1, 그렇지 않을 경우 부정(Negative) 0
    if sentiment >= 0:
        return 1
    
    return 0

In [24]:
df = get_dataset()

Downloading...
From: https://drive.google.com/uc?id=1fOpIndzsAwzrBbuJMjov_alDmgGUsP0d
To: C:\Users\bigne\OneDrive\WorkJ\Machine Learning\NLP\TrainData.tsv
33.6MB [00:00, 36.0MB/s]


In [25]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag

def swn_polarity(text):
    # 감성 지수 초기화 
    sentiment = 0.0
    tokens_count = 0
    
    lemmatizer = WordNetLemmatizer()
    raw_sentences = sent_tokenize(text)
    # 분해된 문장별로 단어 토큰 -> 품사 태깅 후에 SentiSynset 생성 -> 감성 지수 합산 
    for raw_sentence in raw_sentences:
        # NTLK 기반의 품사 태깅 문장 추출  
        tagged_sentence = pos_tag(word_tokenize(raw_sentence))
        for word , tag in tagged_sentence:
            
            # WordNet 기반 품사 태깅과 어근 추출
            wn_tag = penn_to_wn(tag)
            if wn_tag not in (wn.NOUN , wn.ADJ, wn.ADV):
                continue                   
            lemma = lemmatizer.lemmatize(word, pos=wn_tag)
            if not lemma:
                continue
            # 어근을 추출한 단어와 WordNet 기반 품사 태깅을 입력해 Synset 객체를 생성. 
            synsets = wn.synsets(lemma , pos=wn_tag)
            if not synsets:
                continue
            # sentiwordnet의 감성 단어 분석으로 감성 synset 추출
            # 모든 단어에 대해 긍정 감성 지수는 +로 부정 감성 지수는 -로 합산해 감성 지수 계산. 
            synset = synsets[0]
            swn_synset = swn.senti_synset(synset.name())
            sentiment += (swn_synset.pos_score() - swn_synset.neg_score())           
            tokens_count += 1
    
    if not tokens_count:
        return 0
    
    # 총 score가 0 이상일 경우 긍정(Positive) 1, 그렇지 않을 경우 부정(Negative) 0 반환
    if sentiment >= 0 :
        return 1
    
    return 0


In [26]:
df['preds'] = df['review'].apply( lambda x : swn_polarity(x))

In [27]:
y_target = df['sentiment'].values
preds = df['preds'].values

In [28]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score
from sklearn.metrics import recall_score, f1_score, roc_auc_score
import numpy as np

In [29]:
print(confusion_matrix( y_target, preds))
print("정확도:", np.round(accuracy_score(y_target , preds), 4))
print("정밀도:", np.round(precision_score(y_target , preds),4))
print("재현율:", np.round(recall_score(y_target, preds), 4))

[[7649 4851]
 [3578 8922]]
정확도: 0.6628
정밀도: 0.6478
재현율: 0.7138


VADER lexicon을 이용한 Sentiment Analysis

In [35]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

senti_analyzer = SentimentIntensityAnalyzer()
senti_scores = senti_analyzer.polarity_scores(df['review'][0])

In [37]:
print(senti_scores) 

{'neg': 0.13, 'neu': 0.744, 'pos': 0.126, 'compound': -0.8278}


* 'neg' : 부정
* 'neu' : 중립
* 'pos' : 긍정
* 'compound' : 조합한 감성 지수

In [40]:
def vader_polarity(review, threshold = 0.1):
    analyzer = SentimentIntensityAnalyzer()
    scores = analyzer.polarity_scores(review)
    
    # compound 값에 기반하여 threshold 입력 값보다 크면 1, 아니면 0
    
    agg_score = scores['compound']
    final_sentiment = 1 if agg_score >= threshold else 0
    return final_sentiment

In [41]:
df['vader_preds'] = df['review'].apply(lambda x: vader_polarity(x, 0.1))

In [42]:
y_target = df['sentiment'].values
vader_preds = df['vader_preds'].values

print(confusion_matrix( y_target, vader_preds))
print("정확도:", np.round(accuracy_score(y_target , vader_preds),4))
print("정밀도:", np.round(precision_score(y_target , vader_preds),4))
print("재현율:", np.round(recall_score(y_target, vader_preds),4))

[[ 6781  5719]
 [ 1930 10570]]
정확도: 0.694
정밀도: 0.6489
재현율: 0.8456


In [43]:
df['vader_preds'] = df['review'].apply(lambda x: vader_polarity(x, 0.15))

In [44]:
y_target = df['sentiment'].values
vader_preds = df['vader_preds'].values

print(confusion_matrix( y_target, vader_preds))
print("정확도:", np.round(accuracy_score(y_target , vader_preds),4))
print("정밀도:", np.round(precision_score(y_target , vader_preds),4))
print("재현율:", np.round(recall_score(y_target, vader_preds),4))

[[ 6851  5649]
 [ 1975 10525]]
정확도: 0.695
정밀도: 0.6507
재현율: 0.842


In [45]:
df['vader_preds'] = df['review'].apply(lambda x: vader_polarity(x, 0.08))

In [46]:
y_target = df['sentiment'].values
vader_preds = df['vader_preds'].values

print(confusion_matrix( y_target, vader_preds))
print("정확도:", np.round(accuracy_score(y_target , vader_preds),4))
print("정밀도:", np.round(precision_score(y_target , vader_preds),4))
print("재현율:", np.round(recall_score(y_target, vader_preds),4))

[[ 6752  5748]
 [ 1916 10584]]
정확도: 0.6934
정밀도: 0.6481
재현율: 0.8467


참조1 한국어 감성 분석: https://cyc1am3n.github.io/2018/11/10/classifying_korean_movie_review.html

**Reference**

* <a href='https://github.com/SeWonKwon' ><div> <img src ='https://slid-capture.s3.ap-northeast-2.amazonaws.com/public/image_upload/6556674324ed41a289a354258718280d/964e5a8b-75ad-41fc-ae75-0ca66d06fbc7.png' align='left' /> </div></a>

<br>

* 파이썬 머신러닝 완벽 가이드, 권철민